In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [49]:
df=pd.read_csv('train.csv')

#Ostateczna obróbka w wersji prealpha
for i in range(len(df.columns)):
    print(df.columns[i], df[df.columns[i]].dtypes, df[df.iloc[:,i].isna()].shape[0])
    if (df[df.columns[i]].dtypes=='object'):
        df.loc[df.iloc[:,i].isna(),df.columns[i]]='Nothingface'
    else:
        df.loc[df.iloc[:,i].isna(),df.columns[i]]=1000
    print(df.columns[i], df[df.columns[i]].dtypes, df[df.iloc[:,i].isna()].shape[0])
df=df.loc[:,df.columns[(df.columns!='Name') & (df.columns!='Ticket') & (df.columns!='Cabin')]]

PassengerId int64 0
PassengerId int64 0
Survived int64 0
Survived int64 0
Pclass int64 0
Pclass int64 0
Name object 0
Name object 0
Sex object 0
Sex object 0
Age float64 177
Age float64 0
SibSp int64 0
SibSp int64 0
Parch int64 0
Parch int64 0
Ticket object 0
Ticket object 0
Fare float64 0
Fare float64 0
Cabin object 687
Cabin object 0
Embarked object 2
Embarked object 0


In [241]:
l=math.floor(0.2*df.shape[0])
k=[0]*l
dp=[False]*df.shape[0]

for i in range(l):
    k[i]=np.random.randint(1, df.shape[0])
    while (dp[k[i]]==True):
        k[i]=np.random.randint(1, df.shape[0])
    dp[k[i]]=True

dp2=np.logical_not(dp)

train=df[dp2].copy()
test=df[dp].copy()

In [242]:
def proceed(p):
    partalles=0
    gini=1
    partalles=sum(p)
    for jj in range(classes):
        gini-=(p[jj]*p[jj])/(partalles*partalles)
    return (partalles, gini)

def purify(p):
    for jj in range(classes):
        p[jj]=0


i=0

X=[]
Dat=[train]

Pred=df.columns.get_loc('Survived')
classes=2

while (i<len(Dat) and i<1000):    
    #Wartość Nodesa
    ZZ=Dat[i].groupby([Dat[i].columns[Pred]]).count()[Dat[i].columns[0]]
    ZZ=ZZ*ZZ
    value=(1-sum(ZZ)/(len(Dat[i])*len(Dat[i])))
    #print(value[i])
    
    ij=0; vv=0
    p=[0]*classes
    allp=[0]*classes
    #print('Paar', i, len(Dat[i]))
    
    bjaest=(1,)
    #Wybór najlepszego punktu bicia kategorycznego
    for j in range(len(Dat[i].columns)):
        if (j==Pred):
            continue
        #Liczenie funkcji kosztu
        L=Dat[i].iloc[:,[j,Pred]].copy()
        L.sort_values(by=L.columns[0], inplace=True)
                
        wynne=0
        if(L.iloc[:,0].dtypes=='object'):
            for ii in range(len(L)):
                if (ii!=0 and L.iat[ii,0]!=L.iat[ii-1,0]):
                    partalles, gini=proceed(p)
                    purify(p)
                    wynne=wynne+partalles*gini
                p[L.iat[ii,1]]+=1
            partalles, gini=proceed(p)
            purify(p)
            
            wynne=wynne+partalles*gini
            
            wynne=wynne/len(L)
        
        #Miara bicia liczbowego
        else:
            wynne=10
            for ii in range(len(L)):
                allp[L.iat[ii,1]]+=1
            
            pointofsplit=-1
            for ii in range(len(L)):
                if (ii>0 and L.iat[ii,0]!=L.iat[ii-1,0]):
                    part1, gini1=proceed(p)
                    part2, gini2=proceed(list(pd.Series(allp)-pd.Series(p)))
                    
                    if (wynne>(part1*gini1+part2*gini2)/len(L)):
                        wynne=(part1*gini1+part2*gini2)/len(L)
                        pointofsplit=L.iat[ii,0]
                
                p[L.iat[ii,1]]+=1
                
            purify(p)
            purify(allp)
            
        
        #Znalezienie najlepszego bicia
        if (wynne<bjaest[0]):
            if (L.iloc[:,0].dtypes=='object'):
                bjaest=(wynne, L.columns[0])
            else:
                bjaest=(wynne, L.columns[0], pointofsplit)
    
    
    #Modyfikacja Drzewa
    if (bjaest[0]<value and i<20):
        Dat[i].sort_values(by=bjaest[1])
        #Podział - zmienna kategoryczna
        if (len(bjaest)==2):
            Z=np.unique(Dat[i][bjaest[1]])
            AA=len(Dat)
            Dat[i].sort_values(by=bjaest[1], inplace=True)
            mil=Dat[i].columns.get_loc(bjaest[1])
            lastcut=0
            jeez=[]
            
            for j in range(1, len(Dat[i]), 1):
                if (Dat[i].iat[j,mil]!=Dat[i].iat[j-1,mil]):
                    Dat.append(Dat[i].iloc[lastcut:j,:].copy())
                    jeez.append(Dat[i].iat[j-1,mil])
                    lastcut=j
                    
            Dat.append(Dat[i].iloc[lastcut:,:].copy())
            jeez.append(Dat[i].iat[j-1,mil])
            
            X.append((0, bjaest[1], None, AA, jeez))
            
        #Podział - zmienna numeryczna
        else:
            X.append((0, bjaest[1], bjaest[2], len(Dat)))
            
            Dat.append(Dat[i][Dat[i][bjaest[1]]<bjaest[2]].copy())
            Dat.append(Dat[i][Dat[i][bjaest[1]]>=bjaest[2]].copy())
            #Dat.append(Dat[i][Dat[i][bjaest[1]]<bjaest[2]].copy())
            
    else:
        p=[0]*classes
        maks=0
        for j in range(len(Dat[i])):
            s=Dat[i].iat[j,Pred]
            p[s]+=1
            if (p[s]>p[maks]):
                maks=s
        
        X.append((1, maks, bjaest[0]))
    
    i+=1

In [243]:
summa=0

for i in range(len(test)):
    j=0
    while (1):
        if (X[j][0]==1):
            break
        else:
            Mine=test.iat[i,test.columns.get_loc(X[j][1])]
            if (X[j][2]==None):
                All=X[j][4]
                try:
                    k=pd.Index(All).get_loc(test.iat[i,test.columns.get_loc(X[j][1])])
                except:
                    k=0
                outnode=X[j][3]+k
                j=outnode
            else:
                Myval=test.iat[i,test.columns.get_loc(X[j][1])]
                #print('Revaluation', Myval, X[j][2])
                if (Myval<X[j][2]):
                    j=X[j][3]
                else:
                    j=X[j][3]+1
    summa=summa+abs(X[j][1]-test.iat[i,Pred])
print(1-summa/len(test))

0.8258426966292135


In [86]:
for i in range(len(X)):
    print(i, len(Dat[i]), X[i])

0 713 (0, 'Sex', None, 1, 3)
1 247 (0, 'Pclass', 3, 3, 5)
2 466 (0, 'Age', 4.0, 5, 7)
3 115 (0, 'Embarked', None, 7, 10)
4 132 (0, 'Age', 3.0, 10, 12)
5 449 (0, 'Pclass', 2, 12, 14)
6 17 (0, 'SibSp', 3, 14, 16)
7 20 (0, 'Fare', 15.7417, 16, 18)
8 27 (0, 'PassengerId', 502, 18, 20)
9 68 (0, 'Fare', 21.075, 20, 22)
10 130 (0, 'Fare', 29.0, 22, 24)
11 2 (0, 'PassengerId', 531, 24, 26)
12 355 (0, 'Fare', 56.4958, 26, 28)
13 94 (0, 'Age', 37.0, 28, 30)
14 6 (0, 'Age', 3.0, 30, 32)
15 11 (1, 1)
16 7 (1, 1)
17 13 (0, 'Fare', 14.4542, 32, 34)
18 12 (0, 'Age', 1000.0, 34, 36)
19 15 (1, 1)
20 24 (0, 'PassengerId', 375, 36, 38)
21 44 (0, 'Fare', 11.1333, 38, 40)
22 77 (0, 'Parch', 2, 40, 42)
23 53 (0, 'Fare', 28.7125, 42, 44)
24 1 (1, 1)
25 1 (1, 0)
26 15 (0, 'Fare', 69.55, 44, 46)
27 340 (0, 'Age', 9.0, 46, 48)
28 62 (0, 'PassengerId', 588, 48, 50)
29 32 (0, 'PassengerId', 391, 50, 52)
30 1 (1, 1)
31 5 (1, 0)
32 8 (0, 'Parch', 2, 52, 54)
33 5 (1, 1)
34 7 (0, 'PassengerId', 574, 54, 56)
35 5 (1, 

In [ ]:
kgbo=[0.1,0.2,0.3]
kgb=[0.9,0.6,0.7]
print(kgb-kgbo)

In [160]:
print(train)

     PassengerId  Survived  Pclass     Sex      Age  SibSp  Parch      Fare  \
276          277         0       3  female    45.00      0      0    7.7500   
419          420         0       3  female    10.00      0      2   24.1500   
431          432         1       3  female  1000.00      1      0   16.1000   
205          206         0       3  female     2.00      0      1   10.4625   
797          798         1       3  female    31.00      0      0    8.6833   
796          797         1       1  female    49.00      0      0   25.9292   
618          619         1       2  female     4.00      2      1   39.0000   
772          773         0       2  female    57.00      0      0   10.5000   
198          199         1       3  female  1000.00      0      0    7.7500   
24            25         0       3  female     8.00      3      1   21.0750   
11            12         1       1  female    58.00      0      0   26.5500   
759          760         1       1  female    33.00 

In [185]:
print(train.loc[0,:])
print(test.loc[0,:])

PassengerId       1
Survived          0
Pclass            3
Sex            male
Age              22
SibSp             1
Parch             0
Fare           7.25
Embarked          S
Name: 0, dtype: object
PassengerId       1
Survived          0
Pclass            3
Sex            male
Age              22
SibSp             1
Parch             0
Fare           7.25
Embarked          S
Name: 0, dtype: object
